In [1]:
import os
import ssl

import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from config import Config, Device
from datasets import MRIDataset, BalancedMRIDataset
from models import UNetResnet
from trainer import Trainer
from tester import Tester

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = Device.device
print(device)

mps


In [3]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
mean = Config.mean # mean of the entire datasaet
std = Config.std # std of the entire dataaset
image_size = 224

In [4]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=0.5, std=0.5)
])

In [5]:
train_dataset = MRIDataset(
    data_path,
    labels_path,
    split='train',
    transform=transforms,
    max_slices=20
)

val_dataset = MRIDataset(
    data_path,
    labels_path,
    split='val',
    transform=transforms,
    max_slices=20
)

test_dataset = MRIDataset(
    data_path,
    labels_path,
    split='test',
    transform=transforms,
    max_slices=20
)

train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=16)
test_dl = DataLoader(test_dataset, batch_size=16)

In [6]:
ssl._create_default_https_context = ssl._create_stdlib_context

model = UNetResnet().to(device=device)

Using cache found in /Users/fateme/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master
Some weights of ResNetForImageClassification were not initialized from the model checkpoint at BehradG/resnet-18-finetuned-MRI-Brain and are newly initialized because the shapes did not match:
- resnet.embedder.embedder.convolution.weight: found shape torch.Size([64, 3, 7, 7]) in the checkpoint and torch.Size([64, 1, 7, 7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model_name = model.__class__.__name__
model_name

'UNetResnet'

In [8]:
def compute_class_weights_from_csv(csv_file_path):
    # Read the CSV file
    df = pd.read_csv(csv_file_path)

    labels = df['prediction'].values

    # Convert labels to integers if they are not already
    labels = labels.astype(int)

    # Compute class weights
    unique_labels = np.unique(labels)
    class_weights = compute_class_weight(
        class_weight='balanced', classes=unique_labels, y=labels)

    # Convert to torch tensor
    return torch.tensor(class_weights, dtype=torch.float)


# Path to your CSV file
class_weights = compute_class_weights_from_csv(labels_path)

# For binary classification, use the appropriate class weight
# Assuming binary classification with class labels 0 and 1
class_weights = class_weights[1]  # Adjust if necessary
print("Class Weights:", class_weights)

Class Weights: tensor(4.0051)


In [9]:
# Dice Loss definition
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        inputs = torch.sigmoid(inputs)  # Apply sigmoid to ensure inputs are in range [0, 1]
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + self.smooth) / (inputs.sum() + targets.sum() + self.smooth)
        
        return 1 - dice

# Combined Dice + Weighted BCE Loss
class CombinedLoss(nn.Module):
    def __init__(self, class_weights=None, bce_weight=0.5, dice_weight=0.5, smooth=1e-6):
        super(CombinedLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=class_weights)  # Weighted BCE
        self.dice_loss = DiceLoss(smooth=smooth)                 # Dice Loss
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, inputs, targets):
        bce = self.bce_loss(inputs, targets)
        dice = self.dice_loss(inputs, targets)
        
        # Weighted combination of both losses
        loss = self.bce_weight * bce + self.dice_weight * dice
        return loss

In [10]:
criterion = CombinedLoss(class_weights=class_weights).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [11]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=80,
    patience=20,
    threshold=0.5,
    save_path=f"saved_models/{model_name}.pth"
)

# Start training
trainer.train()

Epoch 1/1


100%|██████████| 118/118 [10:18<00:00,  5.24s/it]


Train Loss: 1982.9865, Train Accuracy: 0.7697
Val Loss: 732.4527, Val Accuracy: 0.5032
Precision: 0.1205, Recall: 0.4744, AUC: 0.4908, Avg Metric: 0.3619
Confusion Matrix:
[[278 270]
 [ 41  37]]


In [12]:
model.load_state_dict(torch.load(f"saved_models/{model.__class__.__name__}_best.pth"))

/var/folders/gf/rw5sv1qs443ggnx74bgbk3tc0000gn/T/ipykernel_14768/763035412.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"saved_model

<All keys matched successfully>

In [13]:
tester = Tester(
    model=model,
    criterion = criterion,
    test_dl=test_dl,
    test_dataset=test_dataset,
    device=device,
    threshold=0  # Set the threshold for binary classification
)

# Perform testing and print metrics
tester.test()

Test Loss: 731.2560, Test Accuracy: 0.1246
Precision: 0.1246, Recall: 1.0000, AUC: 0.5000, Avg Metric: 0.5415
Confusion Matrix:
[[  0 548]
 [  0  78]]
